In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflowjs as tfjs
from sklearn.metrics import confusion_matrix

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.9.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


# Descargar los datos

In [2]:
print(f'data imdb')
#descargar todos, pero es muy pesado :(
#(train_data, train_labels), (test_data, test_labels) = tfds.load(name="imdb_reviews", split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True,num_words=10000)


from tensorflow import keras
data=keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000)
print(len(train_data), len(test_data))  
#print(f'{train_data[0]}-{train_labels[0]}')
#print(f'{test_data[0]}-{test_labels[0]}')


data imdb
25000 25000


# Explroar los datos

In [3]:
# 0| negativa
# 1| positiva
print(train_data[0])
print(train_labels[0])


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [4]:
#index the words 
word_index = data.get_word_index() #get the word index
#print(word_index)#tuple of (word, index)
word_index = {k:(v+3) for k,v in word_index.items()} #add 3 to all the values
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index=dict([(value,key) for (key,value) in word_index.items()]) #voltear de key a value a value key

#print(len(train_data[0])-len(train_data[2]))
train_data=keras.preprocessing.sequence.pad_sequences(
    train_data, 
    value=word_index["<PAD>"],
    padding="post",
    maxlen=256
    )
test_data=keras.preprocessing.sequence.pad_sequences(
    test_data, 
    value=word_index["<PAD>"],
    padding="post",
    maxlen=256
    )
#print(len(train_data[0])-len(train_data[2]))

def decode_review(text):
    return " ".join([reverse_word_index.get(i,"?") for i in text])


In [5]:
print(decode_review(train_data[0]))


<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNKNOWN> is an amazing actor and now the same being director <UNKNOWN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNKNOWN> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNKNOWN> to the two little boy's that played the <UNKNOWN> of norman and paul they were just brilliant children are often left out of the <UNKNOWN> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and s

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,16),#los vectores creo
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'), #hidden layer
    #tf.keras.layers.Dropout(0.2),#dropout layer
    tf.keras.layers.Dense(1, activation='sigmoid') #outputs probability between 0 and 1  
    #tf.keras.layers.Dropout(0.2),#dropout layer
    #tf.keras.layers.Dense(1, activation='softmax') #outputs probability between 0 and 1 
])
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])
              #train(adjust) the model
model.fit(train_data, train_labels, epochs=20)

#evaluar la exactitud
test_loss,test_acc=model.evaluate(test_data, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc:.2f}')      

model.summary()

Epoch 1/20
782/782 [==============================] - 5s 6ms/step - loss: 0.5061 - accuracy: 0.7688
Epoch 2/20
782/782 [==============================] - 4s 5ms/step - loss: 0.2640 - accuracy: 0.8971
Epoch 3/20
782/782 [==============================] - 3s 4ms/step - loss: 0.2086 - accuracy: 0.9205
Epoch 4/20
782/782 [==============================] - 4s 5ms/step - loss: 0.1745 - accuracy: 0.9367
Epoch 5/20
782/782 [==============================] - 4s 4ms/step - loss: 0.1499 - accuracy: 0.9462
Epoch 6/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1319 - accuracy: 0.9546
Epoch 7/20
782/782 [==============================] - 4s 5ms/step - loss: 0.1149 - accuracy: 0.9606
Epoch 8/20
782/782 [==============================] - 4s 5ms/step - loss: 0.1005 - accuracy: 0.9674
Epoch 9/20
782/782 [==============================] - 4s 5ms/step - loss: 0.0884 - accuracy: 0.9738
Epoch 10/20
782/782 [==============================] - 4s 5ms/step - loss: 0.0777 - accuracy: 0.9765

In [9]:
#probar
predictions= model.predict(test_data)


def predict(num):
    print(f'Esta\n{decode_review(test_data[num])}')
    print(f'Prediction: {predictions[num]}')
    print(f'Actual:     {test_labels[num]}')

#predict(1)

confusion_matrix(train_data.argmax(axis=1),predictions.argmax(axis=1))

782/782 [==============================] - 2s 2ms/step


array([[ 37,   0,   0, ...,   0,   0,   0],
       [160,   0,   0, ...,   0,   0,   0],
       [170,   0,   0, ...,   0,   0,   0],
       ...,
       [ 25,   0,   0, ...,   0,   0,   0],
       [ 29,   0,   0, ...,   0,   0,   0],
       [ 29,   0,   0, ...,   0,   0,   0]], dtype=int64)

# Guardar y cargar

In [94]:
tfjs.converters.save_keras_model(model, "./modelo2/")#para js
model.save("./modelo2/model.h5")#para python

In [95]:
model=tf.keras.models.load_model("./modelo2/model.h5")#cargar

In [96]:
def review_encode(palabra):
    encoded=[1]
    for word in palabra:
        if word.lower() in word_index:
            encoded.append(word_index[word.lower()])    
        else: 
            encoded.append(2)
    return encoded

In [139]:
#with open("1.txt", encoding="utf-8") as f:
    #1
prueba="Of all the animation classics from the Walt Disney Company, there is perhaps none that is more celebrated than The Lion King Its acclaim is understandable: this is quite simply a glorious work of art The Lion King gets off to a fantastic start. The film's opening number, The Circle of Life, is outstanding. The song lasts for about four minutes, but from the first sound, the audience is floored. Not even National Geographic can capture something this beautiful and dramatic. Not only is this easily the greatest moment in film animation, this is one of the greatest sequences in film history. The story that follows is not as majestic, but the film has to tell a story. Actually, the rest of the film holds up quite well. The story takes place in Africa, where the lions rule. Their king, Mufasa (James Earl Jones) has just been blessed with a son, Simba (Jonathan Taylor Thomas), who goes in front of his uncle Scar (Jeremy Irons) as next in line for the throne. Scar is furious, and sets in motion plans to usurp the throne for himself. After a tragedy occurs and Mufasa is killed, Scar persuades Simba to flee, leaving himself as king. Simba grows up in exile, but he learns that while he can run away from his past, he can never escape it. When viewing the film, it is obvious that The Lion King is quite different from its predecessors (and successors)"
prueba="a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances the sandy dennis character apartment but the film builds to a disturbing climax the characters create an atmosphere with sexual tension and psychological it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy has a small part the moody set fits the content of the story very well in short this movie is a powerful study of loneliness sexual and desperation be patient up the atmosphere and pay attention to the wonderfully written script i praise robert altman this is one of his many films that deals with unconventional fascinating subject matter this film is disturbing but it's sincere and it's sure to a strong emotional response from the viewer if you want to see an unusual film some might even say bizarre this is worth the time unfortunately it's very difficult to find in video stores you may have to buy it off the internet"
inicial=prueba
prueba=prueba.replace(".", "").replace(",", "").replace(":", "").replace("(", "").replace(")", "").split(" ")

prueba=review_encode(prueba)
prueba=keras.preprocessing.sequence.pad_sequences(
    [prueba], 
    value=word_index["<PAD>"],
    padding="post",
    maxlen=256
    )

print(prueba)
#predict=model.predict(test_data[0:2])
predict=model.predict(prueba)
#predict=model.predict(test_data)
# print(inicial)
print(predict)


[[   1    6  176    7 5063   88   12 2679   23 1310    5  109  943    4
   114    9   55  606    5  111    7    4  139  193  273   23    4  172
   270   11 7216    4 8463 2801  109 1603   21    4   22 3861    8    6
  1193 1330    4  105  987   35  841   19  861 1074    5 1987   45   55
   221   15  670 5304  526   14 1069    4  405    5 2438    7   27   85
   108  131    4 5045 5304 3884  405    9 3523  133    5   50   13  104
    51   66  166   14   22  157    9    4  530  239   34 8463 2801   45
   407   31    7   41 3778  105   21   59  299   12   38  950    5 4521
    15   45  629  488 2733  127    6   52  292   17    4 6936  185  132
  1988 5304 1799  488 2693   47    6  392  173    4 4378  270 2352    4
  1500    7    4   65   55   73   11  346   14   20    9    6  976 2078
     7 5293  861    5 4182   30 3127   56    4  841    5  990  692    8
     4 1669  398  229   13 2822  670 5304   14    9   31    7   27  111
   108   15 2033   19 7836 1429  875  551   14   22    9 1193   